# Basic gym Environment and Data

In [2]:
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiDiscrete, MultiBinary, flatdim, flatten_space, unflatten 

import numpy as np
import sys
import pandas as pd
import random
from collections import OrderedDict

In [107]:
class VPPBiddingEnv(Env):
    
    def __init__(self,
                 renewables_df, 
                 bids_df,
                 market_result_df,
                 tenders_df,
                 hist_window_size,
                 forecast_window_size,
                 frame_bound,
                 total_FCR_demand):
        
        # data 
        self.renewables_df = renewables_df
        self.market_result_df = market_result_df
        self.tenders_df = tenders_df
        self.bids_df = bids_df
        self.total_FCR_demand = None
        # TODO: take seasonality out of FCR demand? 
        
        # window_size
        self.hist_window_size = hist_window_size
        self.forecast_window_size = forecast_window_size
        self.frame_bound = frame_bound
        
        self.hydro_df, self.wind_df = self._process_data()
        
        # episode
        self.start_date = pd.to_datetime(self.frame_bound[0])
        self.end_date = pd.to_datetime(self.frame_bound[1])
        self.current_date = self.start_date
        
        self.done = None
        self.total_reward = 0.
        self.total_profit = 0.
        self.history = None
        
        # Slots 
        self.slots_won = [None, None, None, None, None, None]
        self.slot_prices = [None, None, None, None, None, None]
                
        # Spaces
        low = np.float32(np.array([0.0] * 96)) #96 timesteps to min 0.0
        high = np.float32(np.array([1.0] * 96)) #96 timesteps to max 1.0

        self.nested_observation_space = Dict({
            'historic_data': Dict({
                "hydro_historic": Box(low, high, dtype=np.float32),
                "wind_historic":  Box(low, high, dtype=np.float32)
            }),
            'forecast_data':  Dict({
                "hydro_forecast": Box(low, high, dtype=np.float32),
                "wind_forecast": Box(low, high, dtype=np.float32),
                "soc_forecast": Box(low, high, dtype=np.float32)
                # TODO should I keep the Battery state of charge? 
            }),
            'market_data':  Dict({
                "market_demand": Discrete(3), # for the demands 573, 562 and 555 MW
                # TODO for 2021 its always 562, how to handle differetn years? maybe set it as a global constant? 
                # TODO: evtl delete market demand ? 
                "predicted_market_prices":  Box(low=0.0, high=np.float32(1634.52), shape=(6, 1), dtype=np.float32), # for each slot, can be prices of same day last week 
            }),
            'time_features':  Dict({
                "weekday": Discrete(7), # for the days of the week
                "holiday": Discrete(2), # holiday = 1, no holiday = 0
                "month": Discrete(12), # for the month
            }),
            "auction_results":  Dict({
                "slots_won": MultiBinary(5), #boolean for each slot, 0 if loss , 1 if won 
                "slot_prices": Box(low=0.0, high=np.float32(1634.52), shape=(6,), dtype=np.float32)
            })
        })
        
        # first approach: 
        # Slots 1,2,3,4,5,6 = integrated in bidsize if 0 or non 0 
        # bid size: MultiDiscrete([ 25, 25, 25, 25, 25 ]),
        # bid prize: Box(low=0.0, high=1634.52, shape=(6,), dtype=np.float32))

        self.action_space = Tuple((
            # returns array([ 0,  2, 13,  6, 23, 25 ]
            # TODO: realistische Größe eines VPPs rausfinden und VPP Simulation so aufbauen, 
            # TODO: dynamischer Action Space basierend auf den Forecasts? oder soll Agent selbst lernen das niedriger Forecast = keine hohen Angebote
                # geringeres Risiko durch einschränkung des action spaces 
            # TODO: was bedeuten Teilbare angebote?
            # 1 Agent mit action space für mehrere Anlagen 
            # 2 Hierachichal RL: Sub- Agents
            # INFO: TSOs allow divisible and indivisible bids. Indivisible bids can have a maximum bid size of 25 MW in all the participating countries.
            # But divisible bids in 2021 had maximum capacity of 101 MW in one bid (FR)
            MultiDiscrete([ 25, 25, 25, 25, 25 , 25 ]),
            # returns array([1311.5632  ,  665.4643  ,  807.9639  ,  104.337715,  425.967, 205.23262 ]
            Box(low=0.0, high=np.float32(1634.52), shape=(6,), dtype=np.float32)))
        
        # TODO: Add second approach with shares of plants
        '''
        second approach: 
            Share of hydro
            Share of wind
            Share of battery
        '''
    
    
    def reset(self):
        
        self.current_date =  self.current_date + pd.offsets.DateOffset(days=1)
        self.total_FCR_demand = self.tenders_df[str(self.current_date):]["total"][0] 
        print('total_FCR_demand = %s ' % self.total_FCR_demand )
        self.done = False

        # reset for each episode 
        self._get_new_timestamps()
        return self._get_observation()
        
    
    def _get_new_timestamps(self):
        
        self.historic_data_start = self.current_date - pd.offsets.DateOffset(days=1)
        self.historic_data_end =  self.current_date
        
        self.forecast_start = self.current_date + pd.offsets.DateOffset(days=1) # TODO: validate 
        self.forecast_end = self.current_date + pd.offsets.DateOffset(days=2) # TODO: validate
        
        self.market_start = self.current_date
        self.market_end = self.current_date + pd.offsets.DateOffset(hours=20)
        
        self.slot_date_list = []
        
        slot_date = self.current_date
        
        for i in range(0,6):
            self.slot_date_list.append(str(slot_date))
            slot_date = slot_date + pd.offsets.DateOffset(hours=4)   
            
        #print(self.slot_date_list)
    
    
    def _get_observation(self): 
            
        hydro_historic = self.hydro_df[str(self.historic_data_start) : str(self.historic_data_end)].to_numpy()
        wind_historic = self.wind_df[str(self.historic_data_start) : str(self.historic_data_end)].to_numpy()
        
        # TODO: change current date start to 07:00 of the current day, to be shortly before auction time
        hydro_forecast = self.hydro_df[str(self.historic_data_start) : str(self.historic_data_end)].to_numpy()
        wind_forecast =  self.wind_df[str(self.forecast_start) : str(self.forecast_end)].to_numpy()
        
        if self.total_FCR_demand == 573: 
            # in 2020
            market_demand = 0
        if self.total_FCR_demand == 562:
            # in 2021
            market_demand = 1
        if self.total_FCR_demand == 555:
            # in 2021
            market_demand = 2
        else: 
            # default
            market_demand = 1
        # TODO: validate if market demand can be induce in another way, maybe on function call ? 
        
        predicted_market_prices = [10. , 20. , 10. , 20. , 10. , 20.] # TODO: naive prediction: retrieve price of same day last week 
        weekday = random.randrange(7) # TODO: retrieve from time dataframe 
        holiday = random.randrange(2) # TODO: retrieve from time dataframe 
        month = random.randrange(12) # TODO: retrieve from time dataframe 
        
        if self.current_date == (self.start_date + pd.offsets.DateOffset(days=1)):  
            self.slots_won = [0, 0, 0, 0, 0, 0]
            self.slot_prices = [0., 0., 0., 0., 0., 0.]
        
        next_observation = OrderedDict({
            "historic_data": OrderedDict({
                "hydro_historic": hydro_historic,
                "wind_historic": wind_historic
                }),
            "forecast_data": OrderedDict({
                "hydro_forecast": hydro_forecast,
                "wind_forecast": wind_forecast
            }),
            "market_data": OrderedDict({
                "market_demand": market_demand,
                "predicted_market_prices": predicted_market_prices
            }),
            "time_features": OrderedDict({
                "weekday": weekday, 
                "holiday": holiday, 
                "month": month
            }), 
            "auction_results": OrderedDict({
                "slots_won": self.slots_won,
                "slot_prices": self.slot_prices
            })
        })
        
        # TODO: check if next_observation is a valid obervation 
            
        return next_observation
    
    
    def step(self, action):
        
        self._simulate_market(action)
        # calculate reward from state and action 
        step_reward = self._calculate_reward(action)
        self.total_reward += step_reward
        self._update_profit(action)
      
        
        info = dict(
            current_date = str(self.current_date),
            total_reward = self.total_reward,
            total_profit = self.total_profit
        )
        self._update_history(info)
        # TODO: info can contain state variables that are hidden from observations
        # or individual reward terms that are combined to produce the total reward
        
        observation = self._get_observation()
        
        self.done = True
        
        return observation, step_reward, self.done, info
    
    
    def _calculate_reward(self, action):        
        # TODO: validate if capacity could be provided
        # TODO: write reward function
        
        step_reward = 1.0
        return step_reward
    
    
    def _update_profit(self, action):
        trade = False 
        if trade == False: 
            self.total_profit +=0
        if trade == True: 
            self.total_profit = 1
        # TODO: implement profit function 

    
    def _update_history(self, info):
        if not self.history:
            self.history = {key: [] for key in info.keys()}

        for key, value in info.items():
            self.history[key].append(value)

            
    def render(self):
        # TODO: Implement visulisation
        pass
    
    
    def _process_data(self):
        hydro_df = self.renewables_df.loc[:, 'Hydro1']
        wind_df = self.renewables_df.loc[:, 'WP1']
        # TODO: add more power plants
        return hydro_df, wind_df

    
    def _simulate_market(self, action):
        
        # market clearing algorithm:
        
        # for each slot 
        # get all bids
        # bids to dict
        # add bid from action 
        # bring in order by price 
        # accumulate capacities until demand is filled 
        # check if bid is in bid list 
            # if yes, set auciton_won = True and get SETTLEMENTCAPACITY_PRICE
            # if no, set auciton_won = False
        
        
        
        
        
        
        
        ################################################
        
        
        # TODO: Market clearing algorithmus neu schreiben, Angebote aller Länder (ausser DÄNEMARK ?? ) müssen berücksichtigt werden, um gesatm demand zu füllen , erst dann steht preis für slot fest. 
        
        
        
        # for each slot 
        # get all bids
        # bids to dict
        # add bid from action 
        # bring in order by price
        # accumulate capacities until demand is filled  FOR ALL COUNTRIES
            # indivisible flag needs to be included and checked: indivisible offer needs to be fully included
            # check for every country, if Core Portion and export limit are satisfied 
                # for each country 
                    # at least the core portion needs to be satisfied
                    # if capacity is less than total demand, the settlement price for an underfilled country is the price of the last accepted bid
                # at most the export limit needs to be satisfied (all bids for Country - demand of country)
        
        # set prices for all countries
        
        
        ################################################
        
        # The optimisation algorithm calculates the optimal combination of FCR bids to be awarded under consideration of core shares and the maximum exchangeable FCR volumes (export limits of a country) with the goal to reduce total procurement cost of the cooperation. 
        
        # 1. If no export limits or core share constraint are hit, one cross-border marginal price (CBMP) will be determined equalling the most expensive awarded bid in the overall cooperation. 
        # Exceptions from having one CBMP may occur once export limits or core share constraint of one or more countries of the cooperation are hit. In this case, an LMP will be determined based on the local awarded bids within a country.
        
        


        
        current_bids = self.bids_df[self.market_start : self.market_end]
        country_constraints = self.tenders_df[self.market_start : self.market_end]
        accumulated_country_capacities = {
             'DE_capacity': 0,
             'DE_export': 0,
             'DE_core': 0,
             'BE_capacity': 0,
             'BE_export': 0,
             'BE_core': 0,
             'FR_capacity': 0,
             'FR_export': 0,
             'FR_core': 0,
             'NL_capacity': 0,
             'NL_export': 0,
             'NL_core': 0,
             'AT_capacity': 0,
             'AT_export': 0,
             'AT_core': 0,
             'CH_capacity': 0,
             'CH_export': 0,
             'CH_core': 0,
             'SI_capacity': 0,
             'SI_export': 0,
             'SI_core': 0,
             'DK_capacity': 0,
             'DK_export': 0,
             'DK_core': 0}
        
        
        for slot in range(0, len(self.slot_date_list)):
            slot_date = self.slot_date_list[slot]
            print("slot_date = %s" % (slot_date))
            bids_in_slot = current_bids[slot_date : slot_date].reset_index(drop=True).reset_index(drop=False)
            bids_list = bids_in_slot.to_dict('records')
            
            slot_constraints = country_constraints[slot_date : slot_date].reset_index(drop=True).reset_index(drop=False)
            slot_constraints = slot_constraints.to_dict('records')[0]            

            print("bids_list = ")
            print("\n".join(" \t{}".format(k) for k in bids_list))
            
            # get the lenght of the list ot create an index fo the agents bid that now will be added
            agents_bid_index = len(bids_list)
            # extract the bid size out of the agents action
            agents_bid_size = action[0][slot]
            # extract the bid price out of the agents action
            agents_bid_price = action[1][slot]
            print("agents_bid_size = %s" % (agents_bid_size))
            print("agents_bid_price = %s" % (agents_bid_price))
            print("agents_bid_index = %s" % (agents_bid_index))
            # add the selected bid from the agent to the list of all bids
            bids_list.append({'index': agents_bid_index, 'offered': agents_bid_size, 'price': agents_bid_price, "country": "DE", "indivisible": False})
            # sort the list based on the price to later accumulate all bids' capacity (but ordered on price)
            sorted_bids_list_by_price = sorted(bids_list, key=lambda x: x['price'])
            
            print("sorted_bids_list_by_price = ")
            print("\n".join(" \t{}".format(k) for k in sorted_bids_list_by_price))
            
            country_list = list(set([x['country'] for x in sorted_bids_list_by_price]))
            total_accumulated_capacity = 0
            accepted_bids = []

            for bid in sorted_bids_list_by_price:
                # 2.1 Case of hitting a limit constraint
                # It is important to understand that an export limit or core share constraint is hit whenever it influences the solution and not only when the quantity awarded in a country is exactly equal to the respective limit quantity of that country.
                
                country_prefix = bid["country"]
                # add the capacity of the bid to the accumulated capacity of a single country
                accumulated_country_capacities[country_prefix + "_capacity"]
                # add the capacity of each bid to the accumulated capacity
                total_accumulated_capacity += bid["offered"]
                # add the bid to the accepted bids list
                bid["allocated"] = bid["offered"]
                accepted_bids.append(bid)
                print('bid["index"] = %s ' % (bid["index"]))
                print("agents_bid_index = %s" % (agents_bid_index))
                print("total_accumulated_capacity = %s" % (total_accumulated_capacity))
                
                if bid["index"] == agents_bid_index:
                    # set boolean for auction win 
                    self.slots_won[slot] = 1
                if total_accumulated_capacity >= self.total_FCR_demand:
                    # if total_accumulated_capacity is bigger than the demand, the last indvisible offer(s) need to be reduced
                    
                    # 2.1.2 Check if core share of every country is hit
                    for country in country_list: 
                        if accumulated_country_capacities[country + "_core"] < slot_constraints[country + "_core"]: 
                            print("CORE SHARE TOO SMALL FOR COUNTRY: " + country)
                    
                    
                    if total_accumulated_capacity > self.total_FCR_demand: 
                        # get the overfilled capacity (difference)
                        overfilled_capacity = total_accumulated_capacity - self.total_FCR_demand
                        # get list of accepted bids that are divisible (= that is not indivisible)
                        accepted_bids_divisible = [bid for bid in accepted_bids if not bid['indivisible']]
                        # get last accepted bid that is divisible
                        accepted_bids_divisible[-1]
                        
                        #bid["allocated"] =
                        
                    # substract too
                    
                    
                    # last accepted bid sets settlement price of auction
                    settlement_price = bid["price"]
                    # set settlement price for the current auctioned slot in slot_prices list
                    self.slot_prices[slot] = settlement_price
                    
                    print("total_accumulated_capacity = %s" % (total_accumulated_capacity))
                    print("self.slots_won = ")
                    print("\n".join("won: \t{}".format(k) for k in self.slots_won))
                    print("self.slot_prices = ")
                    print("\n".join("price: \t{}".format(k) for k in self.slot_prices))

                    break
                    

In [ ]:
{'index': 0, 'offered': 1, 'price': 0.0, 'country': 'FR', 'indivisible': False}
{'index': 1, 'offered': 1, 'price': 0.0, 'country': 'DE', 'indivisible': False}
{'index': 2, 'offered': 4, 'price': 0.0, 'country': 'BE', 'indivisible': False}
{'index': 3, 'offered': 1, 'price': 0.0, 'country': 'BE', 'indivisible': False}
{'index': 4, 'offered': 3, 'price': 0.0, 'country': 'NL', 'indivisible': False}

[x['value'] for x in list_of_dicts]


### Data Preparation

In [108]:
# Renewables 
renewables_df = pd.read_csv("data/clean/renewables.csv", sep = ";").set_index("time", drop = True)

# Tenders (Demand) 
tenders_df = pd.read_csv("data/clean/tenders_all.csv", sep = ";", index_col = 0).set_index("SLOT_START", drop = True)
tenders_df.index = pd.to_datetime(tenders_df.index)
tenders_df = tenders_df.rename(columns={'TOTAL_DEMAND_[MW]': "total", 'GERMANY_BLOCK_DEMAND_[MW]': "DE_demand",
       'GERMANY_BLOCK_EXPORT_LIMIT_[MW]': "DE_export", 'GERMANY_BLOCK_CORE_PORTION_[MW]': "DE_core",
       'BELGIUM_BLOCK_DEMAND_[MW]': "BE_demand", 'BELGIUM_BLOCK_EXPORT_LIMIT_[MW]': "BE_export",
       'BELGIUM_BLOCK_CORE_PORTION_[MW]': "BE_core", 'FRANCE_BLOCK_DEMAND_[MW]': "FR_demand",
       'FRANCE_BLOCK_EXPORT_LIMIT_[MW]': "FR_export", 'FRANCE_BLOCK_CORE_PORTION_[MW]': "FR_core",
       'NETHERLANDS_BLOCK_DEMAND_[MW]': "NL_demand", 'NETHERLANDS_BLOCK_EXPORT_LIMIT_[MW]': "NL_export",
       'NETHERLANDS_BLOCK_CORE_PORTION_[MW]': "NL_core", 'AUSTRIA_BLOCK_DEMAND_[MW]': "AT_demand",
       'AUSTRIA_BLOCK_EXPORT_LIMIT_[MW]': "AT_export", 'AUSTRIA_BLOCK_CORE_PORTION_[MW]': "AT_core",
       'SWITZERLAND_BLOCK_DEMAND_[MW]': "CH_demand", 'SWITZERLAND_BLOCK_EXPORT_LIMIT_[MW]': "CH_export",
       'SWITZERLAND_BLOCK_CORE_PORTION_[MW]': "CH_core", 'SLOVENIA_BLOCK_DEMAND_[MW]': "SI_demand",
       'SLOVENIA_BLOCK_EXPORT_LIMIT_[MW]': "SI_export", 'SLOVENIA_BLOCK_CORE_PORTION_[MW]': "SI_core",
       'DENMARK_BLOCK_DEMAND_[MW]': "DK_demand", 'DENMARK_BLOCK_EXPORT_LIMIT_[MW]': "DK_export",
       'DENMARK_BLOCK_CORE_PORTION_[MW]': "DK_core"})
#TODO: outsourcing of data cleaning to data cleaning notebook 

# Bids (Offers)
bids_df = pd.read_csv("data/clean/bids_all.csv", sep = ";", index_col = 0).set_index("SLOT_START", drop = True)
bids_df.index = pd.to_datetime(bids_df.index)
bids_df["indivisible"] = bids_df['NOTE'].str.contains(r'INDIVISIBLE', na=False)
bids_df = bids_df.rename(columns={'OFFERED_CAPACITY_PRICE_[EUR/MW]': 'price', 'OFFERED_CAPACITY_[MW]': 'offered', "COUNTRY" : "country"})
bids_df["offered"] = bids_df["offered"].astype(int)
bids_df = bids_df[["offered", "price", "country", "indivisible"]]

# maybe-TODO: check if structure of Bids-Dataframe is correct or can be optimized? 
    # date # slot = 1,2,3,4,5,6

# Market Results
market_result_df = []

hist_window_size = 1 # in days
forecast_window_size = 1 # in days
start_index = "2021-01-01 00:00:00+00:00"
end_index = "2021-12-30 00:00:00+00:00"
frame_bound = (start_index, end_index)

### Initilize Environment

In [109]:
env = VPPBiddingEnv(renewables_df,
                    bids_df,
                    market_result_df,
                    tenders_df,
                    hist_window_size,
                    forecast_window_size,
                    frame_bound,
                    total_FCR_demand)

### Run Episodes

In [115]:
demand = [{'index': 0, 'total': 1409.0, 'DE_demand': 562.0, 'DE_export': 168.0, 'DE_core': 169.0, 'BE_demand': 87.0, 'BE_export': 100.0, 'BE_core': 27.0, 'FR_demand': 508.0, 'FR_export': 152.0, 'FR_core': 153.0, 'NL_demand': 114.0, 'NL_export': 100.0, 'NL_core': 35.0, 'AT_demand': 71.0, 'AT_export': 100.0, 'AT_core': 22.0, 'CH_demand': 67.0, 'CH_export': 100.0, 'CH_core': 21.0, 'SI_demand': "nan", 'SI_export': "nan", 'SI_core': "nan", 'DK_demand': "nan", 'DK_export': "nan", 'DK_core': "nan"}]
demand = demand[0]
demand["DE_demand"]

562.0

In [116]:
demand

{'index': 0,
 'total': 1409.0,
 'DE_demand': 562.0,
 'DE_export': 168.0,
 'DE_core': 169.0,
 'BE_demand': 87.0,
 'BE_export': 100.0,
 'BE_core': 27.0,
 'FR_demand': 508.0,
 'FR_export': 152.0,
 'FR_core': 153.0,
 'NL_demand': 114.0,
 'NL_export': 100.0,
 'NL_core': 35.0,
 'AT_demand': 71.0,
 'AT_export': 100.0,
 'AT_core': 22.0,
 'CH_demand': 67.0,
 'CH_export': 100.0,
 'CH_core': 21.0,
 'SI_demand': 'nan',
 'SI_export': 'nan',
 'SI_core': 'nan',
 'DK_demand': 'nan',
 'DK_export': 'nan',
 'DK_core': 'nan'}

In [111]:
#episodes = 365
episodes = 1
score = 0


for episode in range(1, episodes+1):
    print('Start of Episode:{} '.format(episode))
    observation = env.reset()
    
    # timestep defined as: 1 step = 1 day.
    for timestep in range(1):
        #env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        score+=reward
        if done:
            print('Episode:{} Score:{} Info:{}'.format(episode, score, info))
            break
env.close()

Start of Episode:1 
total_FCR_demand = 1409.0 
slot_date = 2021-01-03 00:00:00+00:00
[{'index': 0, 'total': 1409.0, 'DE_demand': 562.0, 'DE_export': 168.0, 'DE_core': 169.0, 'BE_demand': 87.0, 'BE_export': 100.0, 'BE_core': 27.0, 'FR_demand': 508.0, 'FR_export': 152.0, 'FR_core': 153.0, 'NL_demand': 114.0, 'NL_export': 100.0, 'NL_core': 35.0, 'AT_demand': 71.0, 'AT_export': 100.0, 'AT_core': 22.0, 'CH_demand': 67.0, 'CH_export': 100.0, 'CH_core': 21.0, 'SI_demand': nan, 'SI_export': nan, 'SI_core': nan, 'DK_demand': nan, 'DK_export': nan, 'DK_core': nan}]
bids_list = 
 	{'index': 0, 'offered': 1, 'price': 0.0, 'country': 'FR', 'indivisible': False}
 	{'index': 1, 'offered': 1, 'price': 0.0, 'country': 'DE', 'indivisible': False}
 	{'index': 2, 'offered': 4, 'price': 0.0, 'country': 'BE', 'indivisible': False}
 	{'index': 3, 'offered': 1, 'price': 0.0, 'country': 'BE', 'indivisible': False}
 	{'index': 4, 'offered': 3, 'price': 0.0, 'country': 'NL', 'indivisible': False}
 	{'index': 5, 

In [3]:
accepted_bids = [{'index': 0, 'offered': 11, "allocated" : 10, 'price': 0.0, 'country': 'DE', 'indivisible': False}
,{'index': 1, 'offered': 1, "allocated" : 10,'price': 0.0, 'country': 'DE', 'indivisible': True}
,{'index': 2, 'offered': 1,"allocated" : 10, 'price': 0.0, 'country': 'FR', 'indivisible': True}
,{'index': 3, 'offered': 4,"allocated" : 10, 'price': 0.0, 'country': 'DE', 'indivisible': False}]

country_tenders = [{"DE": 
                    {"total": 0, "export": 11, "core" : 10},
                    "BE": 
                    {"total": 0, "export": 11, "core" : 10}}]

country_capacity =  {"DE": 555, "BE": 200}        

# go through all accepted bids in reversed order sorted by price
for bid in reversed(accepted_bids):
    # if an order is divisible and can be divided...
    if not bid["indivisible"]:
        # check if the offered capacity of the bid is a minimum of 1 bigger than the overfilled_capacity (so it can be substracted)
        if bid["offered"] > overfilled_capacity: 
            difference_to_core = country_tenders[bid["country"]]["core"]
            overfilled_capacity
        bid["allocated"] = bid["allocated"]-1
        
        break

print(accepted_bids)


NameError: name 'overfilled_capacity' is not defined

In [4]:
accepted_bids = [{'index': 0, 'offered': 11, "allocated" : 10, 'price': 0.0, 'country': 'DE', 'indivisible': False}
,{'index': 1, 'offered': 1, "allocated" : 10,'price': 0.0, 'country': 'DE', 'indivisible': True}
,{'index': 2, 'offered': 1,"allocated" : 10, 'price': 0.0, 'country': 'FR', 'indivisible': True}
,{'index': 3, 'offered': 4,"allocated" : 10, 'price': 0.0, 'country': 'DE', 'indivisible': False}]


In [8]:

list(set([x['country'] for x in accepted_bids]))

['FR', 'DE']

In [9]:
display(env.nested_observation_space.sample())

NameError: name 'env' is not defined

In [ ]:
display(env.action_space.sample())

In [ ]:
env.nested_observation_space

In [ ]:
flatdim(env.nested_observation_space)

In [ ]:
flatten_space(env.nested_observation_space)

In [ ]:
# flatten(space, x)
# Flatten a data point from a space.
# This is useful when e.g. points from spaces must be passed to a neural network, which only understands flat arrays of floats.
# Accepts a space and a point from that space. Always returns a 1D array. 

flatten(env.nested_observation_space, env.nested_observation_space.sample())

In [ ]:
# check if flattened data point is in space

flatten((env.nested_observation_space, env.nested_observation_space.sample()) in flatten_space(env.nested_observation_space)

In [ ]:

flattened_datapoint = flatten(env.nested_observation_space, env.nested_observation_space.sample())
unflattened_datapoint = unflatten(env.nested_observation_space, flattened_datapoint)
unflattened_datapoint

In [ ]:
# ----------------------
# other way of representing observation

'''
next_observation = Dict({
    'historic_data': Dict({
        "hydro_historic": Box(low, high, dtype=np.float32)
        "wind_historic":  Box(low, high, dtype=np.float32)
    }),
    'forecast_data':  Dict({
        "hydro_forecast": Box(low, high, dtype=np.float32),
        "wind_forecast": Box(low, high, dtype=np.float32),
        "soc_forecast": Box(low, high, dtype=np.float32)
        # TODO should I keep the Battery state of charge? 
    }),
    'market_data':  Dict({
        "market_demand": Discrete(3), # for the demands 573, 562 and 555 MW
        # TODO for 2021 its always 562, how to handle differetn years? maybe set it as a global constant? 

        "predicted_market_prices":  Box(low=0.0, high=1634.52, shape=(6, 1), dtype=np.float32), # for each slot, can be prices of same day last week 
    }),
    'time_features':  Dict({
        "weekday": Discrete(7), # for the days of the week
        "holiday": Discrete(2), # holiday = 1, no holiday = 0
        "month": Discrete(12), # for the month
    })
})
'''


# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states_hydro_historic = env.nested_observation_space["historic_data"]["hydro_historic"].shape
states_wind_historic = env.nested_observation_space["historic_data"]["wind_historic"].shape

actions = env.action_space[0].shape

In [ ]:
display(states_hydro_historic)
display(states_wind_historic)
display(actions)

In [ ]:
def build_model(states, actions):
    model = Sequential()
    # flatten? 
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model 


In [ ]:
model = build_model(states, actions)


In [ ]:
model.summary()


# 3. Build Agent with Keras-RL


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)


# 4. Reloading Agent from Memory


In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)


In [ ]:
del model
del dqn
del env

In [ ]:
from gym.envs.registration import register

register(
    id='vpp-v0',
    entry_point='gym_foo.envs:FooEnv',
)


In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')


In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)
